# NIDS based on Network Flow Measurements

### What is network flow and flow measurement?

A network flow consists of all network packets that have the same flow identifier (ID). The flow ID can be extracted from the packet header and consists, and is usually defined by the 5-tuple. 

<hr/>
<center>
<img src="images/counting/5tuple.png"/>
</center>

Flow measurement is a collection of flow data. Measuring the flow data is useful in a number of applications such as traffic analysis, network visibility, congestion control, heavy-hitter detection, anomaly detection, and intrusion detection.

Flow-measurement based NIDS use the network flow data (flow size, flow volume, flow features, etc.) for intrusion detection.

### Getting started

In this exercise we take flow size/flow volume as our parameter and we consider measuring/counting the flow size/flow volume of flow IDs to detect an anomaly and intrusion. We define flow size as the number of packets and flow bytes as the byte volume of packets. We start with flow size first and later with flow volume.

#### How do we measure it?
We require a counter array to count the flow size. In software, we can make use of a library to store the flow IDs and corresponding flow sizes. 


#### Load the dataset first

In [2]:
#Load dataset
import import_ipynb
from lib.dataset import NIDSDataset

data_file = 'data/packets.npy'
labels_file = 'data/labels.npy'

dataset = NIDSDataset(data_file, labels_file)

#### Exercise 20.1

In [4]:
# INITIALISE THE COUNTERS TO ZERO

wordcounter=0
framecounter=0
flowid = ""
flowvolume = ""
library = {} # Library to store the flowid and sizes. Here we are taking only flow sizes (i.e; the size of
             # each packet is taken as 1)

# loop over all datasets
for d in dataset:

    decision_is_made = 0 # decision_is_made = 1 when ethertype is not 0x0800 or packet is neither TCP nor UDP
                         # decision_is_made = 2 when the flow ID is extracted
    wordcounter = 0
    flowid = ""
    flowvolume = "" 

    # loop over all words
    for word in d:
        # stop parsing if a decission is made
        if decision_is_made == 0:

            # assert Ethertype is 0x0800 - in link layer header
            if wordcounter == 3:
                if(word[0] == 8) and (word[1] == 0):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
            # assert proto is tcp or udp 6/17 - in network layer header
            if wordcounter == 5:
                if(word[3] == 6)or(word[3] == 17):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
                    
            # examine Total length - in network layer header
            if(wordcounter == 4):
                flowvolume += hex(word[0])[2:] # ip len 1/2
                flowvolume += hex(word[1])[2:]  # ip len 2/2
                
            # examine Source Address - in network layer header
            if wordcounter == 6:
                flowid += hex(word[2])[2:]  # ip SA 1/4
                flowid += hex(word[3])[2:]  # ip SA 2/4

            # examine Destination Address - in network layer header
            if wordcounter == 7:
                flowid += hex(word[0])[2:]  # ip SA 3/4
                flowid += hex(word[1])[2:]  # ip SA 4/4
                flowid += hex(word[2])[2:]  # ip DA 1/4
                flowid += hex(word[3])[2:]  # ip DA 2/4

            if wordcounter == 8:
                flowid += hex(word[0])[2:]  # ip DA 3/4
                flowid += hex(word[1])[2:]  # ip DA 4/4
                flowid += hex(word[2])[2:]  # ip SPort 1/2
                flowid += hex(word[3])[2:]  # ip SPort 2/2

            # examine Destination port - in transport layer header
            if wordcounter == 9:
                flowid += hex(word[0])[2:]  # ip DPort 1/2
                flowid += hex(word[1])[2:]  # ip DPort 2/2
                decision_is_made = 2
                break
                # TODO here you should break out of the for loop that
                #      iterates over the words
        # print(word, end='')
        wordcounter += 1
        
    flowvolume = (int("0x"+flowvolume,16))
    
    if(decision_is_made==2):
        """ WRITE CODE """
        #check if the flowid is in the library and
        # if it is present increment flow size by 1, and if not present add the new 
        # flowid with a size 1. 
        if flowid in library:
            library[flowid] += flowvolume
        else:
            library[flowid] = flowvolume

    # end of iteration over words

# end of iteration over datasets

# PRINTS all the flowids along with the sizes
for flowid in library:  
    print(flowid, '->', library[flowid])


c0a8a32c0a8a3db2ccc4 -> 4550
c0a8ac346d1ca6301f90 -> 540
346d1cc0a8ac1f90a630 -> 60
c0a8a32ac10010509eac -> 352
c0a8a32ac10010509eae -> 240
c0a8a32ac1001050c1f2 -> 364
c0a8a32ac1001050c9e4 -> 520
c0a8a32ac1001050c9e6 -> 572
c0a8a32ac1001050c9e7 -> 520
c0a8a32ac1001050c9e8 -> 572
c0a8a32ac1001050c9e9 -> 624
c0a8a32ac1001050c9eb -> 624
c0a8a32ac1001050c9ec -> 624
c0a8a32ac1001050c17d -> 312
c0a8a32ac1001050c17e -> 312
c0a8a32ac1001050c17f -> 156


#### Do any of the flows exhibits anomalous behaviour?

#### Exercise 20.2

we set a threshold to determine which of the flowids exhibits anomalous behaviour by exceeding the allocated bandwidth.

In [51]:
threshold = 600

print("The following flowids exceeds the threshold:")
""" WRITE code """
# print all the flowids from the librabry that exceeds the threshold
for flowid in library:
    if(library[flowid]>threshold):
        print(flowid, '->', library[flowid])

The following flowids exceeds the threshold:
c0a8a32c0a8a3db2ccc4 -> 4550
c0a8a32ac1001050c9e9 -> 624
c0a8a32ac1001050c9eb -> 624
c0a8a32ac1001050c9ec -> 624


#### Making use of a hashtable
If we want to reduce the memory footprint, we can hash the flowids to locate an index to store the flowids and flowsize. Python actually can make a hashtable of it, where the flow IDs and associated flow size are stored as key-value pairs. In HDL, there are no such concept as hashtables, so we have to implement the hash tables manually.

<hr/>
<center>
<img src="images/counting/hashmap.png"/>
</center>

#### How to address hash collisions?
Since we are trimming down the hashed value according to the length of the hash table, there will be collisions. We need to take measures such as chaining to avoid hash collisions in hashtables. C

Chaining is simple and is like a linked list, where each index can include a separate list with many elements. And the advantage is that hash table never fills up, we can always add more elements to the chain.

<hr/>
<center>
<img src="images/counting/chaining.png"/>
</center>

#### Exercise 20.3

In [6]:
import array
import hashlib

n = 256 # length of the hashtable
array = [None] * n

""" Define functions here for the hashtable """

def _hash(key):
    """ Md5 hash function to calculate the index"""
    n=256
    md5 = hashlib.md5(str(hash(key)).encode('utf-8'))
    return int(md5.hexdigest(), 16) % n
    
def add_ht(key, value):
    """Add a value to hashtable by its key and update the contents if the cell is not empty"""
    index = _hash(key)
    if array[index] is not None:  # This index already contain some values.
        # WRITE the code to Check if the flowid present in the [key,value] pair in the array[index] is equal to 
        # the incoming flowid. if equal, then add the flow size to the value.
        # If the flowids are not equal, then we have to think of the chaining and append the new
        # element to the list in the array[index].
        for kvp in array[index]:  
            if kvp[0] == flowid:  
                kvp[1] = kvp[1]+value 
                break 
        else:     
            array[index].append([flowid, value])  
    else: 
        # WRITE the code here
        # If the index is empty, creare an empty list in the array[index] and append the key-value pair to the list.
        array[index] = []
        array[index].append([flowid, value])
        
def query_ht(key):
    """Get a value by key"""
    index = _hash(key)
    if array[index] is None:
        return 0
    else:
        """ WRITE code """
        # Loop through all key-value-pairs and find if the flowid exist. 
        # If exists then return its value. # If no return was done during loop, 
        # it means flowid does not exist and return 0
        for kvp in array[index]:
            if kvp[0] == key:
                return kvp[1]
        return 0
        

In [7]:
""" Now, again reading the dataset to update the hashtable. Copy and paste the code """
""" from the Exercise 20.1 """

wordcounter=0
framecounter=0
flowid = ""     # flow id
flowvolume = "" # flow volume
flowlist = []  # keeps a list to store flows

# loop over all datasets
for d in dataset:

    decision_is_made = 0 # decision_is_made = 1 when ethertype is not 0x0800 or packet is neither TCP nor UDP
                         # decision_is_made = 2 when the flow ID is extracted
    wordcounter = 0
    flowid = ""
    flowvolume = ""

    # loop over all words
    for word in d:
        # stop parsing if a decission is made
        if decision_is_made == 0:

            # assert Ethertype is 0x0800 - in link layer header
            if wordcounter == 3:
                if(word[0] == 8) and (word[1] == 0):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
            # assert proto is tcp or udp 6/17 - in network layer header
            if wordcounter == 5:
                if(word[3] == 6)or(word[3] == 17):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
                    
            # examine Total length - in network layer header
            if(wordcounter == 4):
                flowvolume += hex(word[0])[2:] # ip len 1/2
                flowvolume += hex(word[1])[2:]  # ip len 2/2
                
            # examine Source Address - in network layer header
            if wordcounter == 6:
                flowid += hex(word[2])[2:]  # ip SA 1/4
                flowid += hex(word[3])[2:]  # ip SA 2/4

            # examine Destination Address - in network layer header
            if wordcounter == 7:
                flowid += hex(word[0])[2:]  # ip SA 3/4
                flowid += hex(word[1])[2:]  # ip SA 4/4
                flowid += hex(word[2])[2:]  # ip DA 1/4
                flowid += hex(word[3])[2:]  # ip DA 2/4

            if wordcounter == 8:
                flowid += hex(word[0])[2:]  # ip DA 3/4
                flowid += hex(word[1])[2:]  # ip DA 4/4
                flowid += hex(word[2])[2:]  # ip SPort 1/2
                flowid += hex(word[3])[2:]  # ip SPort 2/2

            # examine Destination port - in transport layer header
            if wordcounter == 9:
                flowid += hex(word[0])[2:]  # ip DPort 1/2
                flowid += hex(word[1])[2:]  # ip DPort 2/2
                decision_is_made = 2
                break
                # TODO here you should break out of the for loop that
                #      iterates over the words
        # print(word, end='')
        wordcounter += 1
        
    flowvolume = (int("0x"+flowvolume,16))
        
    if flowid not in flowlist:
        flowlist.append(flowid)

    # f1 = 'c0a8ac346d1ca6301f90'
    # f2 = '346d1cc0a8ac1f90a630'
    # f3 = 'c0a8a32ac10010509eac'


    """ Updating the table """
    # Here we are taking the actual flow volume to update
    """WRITE code"""
    # add flowid and flowvolume to the hashtable by calling the function
    add_ht(flowid,flowvolume)


#### Checking for anomalies

In [8]:
"""Print the flow IDs that exceeds a threshold"""
threshold = 600
"""WRITE code"""
# iterate through the flowlist and print those flowids having total volume greater than the threshold
for flowid in flowlist:
    volume = query_ht(flowid)
    if(volume>threshold):
        print(flowid, '->', volume)

c0a8a32c0a8a3db2ccc4 -> 4550
c0a8a32ac1001050c9e9 -> 624
c0a8a32ac1001050c9eb -> 624
c0a8a32ac1001050c9ec -> 624


### Is this really hardware friendly?

Even though hash tables and collision resistant mechanisms such as linked lists is a nice solution, linked lists are not tailored for hardware.  Also, one-to-one mapping consumes considerable amount memory as the number of flows encountered could be in millions. 

There we have sketches, which are hardware friendly and at the same time has some inbuilt collision resistance mechanisms. Instead of one-to-one mapping, one flow is mapped to multiple counters and each counter is shared by multiple flows in the case of collisions. Requires less memory as we don't have to store the flowids, but of course at the cost of a possibility of overestimation.

### CM Sketch

CM sketch is represented by a 2-d array of counters with width w and depth d. Depth is the number of hash functions used.

<hr/>
<center>
<img src="images/counting/CM_sketch.png"/>
</center>

h1, h2,...,hd are independant hash values of the flowid f1. Each flowid is mapped to d counters in the sketch during update and minimum of all the d counters are the result during a query.

#### Exercise 20.4a

In [88]:
import array
import hashlib
""" CM Sketch parameters and functions  """    
m = 16
d = 4

#initializes an empty 2-d array
tables = []
for i in range(d):
    table = array.array("l", (0 for i in range(m))) # "l" is the typecode indicating that the type is signed long
    tables.append(table)
    
# Here, instead of using d independant hashes, 
# the output of md5 hash is split into d hash values.
def _hash(flowid):
    """ hash computation """
    m=16
    d=4
    md5 = hashlib.md5(str(hash(flowid)).encode('utf-8'))
    for i in range(d):
        md5.update(str(i).encode('utf-8'))
        yield int(md5.hexdigest(), 16) % m # yield gives a generator object and has to be iterated to read the values.
        
def add_cms(flowid, value):
    threshold = 600
    """Add a value to hashtable by its key and update the contents if the cell is not empty"""
    # get the d indexed locations of the sketch
    indices = _hash(flowid)
    # Iterate through tables and indices and update the value stored in each indexed location
    # Whether the location is empty or not, just add the value to the already existing value
    for table, i in zip(tables,indices):
        """ WRITE Code to update the value in the indexed location (1 line of code)"""
        table[i] += value

def query_cms(flowid):
    """Get a value by key"""
    indices = _hash(flowid)
    """ WRITE Code """
    # Iterate through tables and indices and return the minimum of the values stored in the indexed locations
    return min(table[i] for table, i in zip(tables, indices))


# f1 = 'c0a8ac346d1ca6301f90'

""" Now, again reading the dataset to update the sketch. Copy and paste the code """
""" from the Exercise 20.1 """

wordcounter=0
framecounter=0
flowid = ""     # flow id
flowvolume = "" # flow volume
flowlist = []  # keeps a list to store flows

# loop over all datasets
for d in dataset:

    decision_is_made = 0 # decision_is_made = 1 when ethertype is not 0x0800 or packet is neither TCP nor UDP
                         # decision_is_made = 2 when the flow ID is extracted
    wordcounter = 0
    flowid = ""
    flowvolume = ""

    # loop over all words
    for word in d:
        # stop parsing if a decission is made
        if decision_is_made == 0:

            # assert Ethertype is 0x0800 - in link layer header
            if wordcounter == 3:
                if(word[0] == 8) and (word[1] == 0):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
            # assert proto is tcp or udp 6/17 - in network layer header
            if wordcounter == 5:
                if(word[3] == 6)or(word[3] == 17):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
                    
            # examine Total length - in network layer header
            if(wordcounter == 4):
                flowvolume += hex(word[0])[2:] # ip len 1/2
                flowvolume += hex(word[1])[2:]  # ip len 2/2
                
            # examine Source Address - in network layer header
            if wordcounter == 6:
                flowid += hex(word[2])[2:]  # ip SA 1/4
                flowid += hex(word[3])[2:]  # ip SA 2/4

            # examine Destination Address - in network layer header
            if wordcounter == 7:
                flowid += hex(word[0])[2:]  # ip SA 3/4
                flowid += hex(word[1])[2:]  # ip SA 4/4
                flowid += hex(word[2])[2:]  # ip DA 1/4
                flowid += hex(word[3])[2:]  # ip DA 2/4

            if wordcounter == 8:
                flowid += hex(word[0])[2:]  # ip DA 3/4
                flowid += hex(word[1])[2:]  # ip DA 4/4
                flowid += hex(word[2])[2:]  # ip SPort 1/2
                flowid += hex(word[3])[2:]  # ip SPort 2/2

            # examine Destination port - in transport layer header
            if wordcounter == 9:
                flowid += hex(word[0])[2:]  # ip DPort 1/2
                flowid += hex(word[1])[2:]  # ip DPort 2/2
                decision_is_made = 2
                break
                # TODO here you should break out of the for loop that
                #      iterates over the words
        # print(word, end='')
        wordcounter += 1
        
    flowvolume = (int("0x"+flowvolume,16))
        
    if flowid not in flowlist:
        flowlist.append(flowid)

    """ Updating the table """
    # Here we are taking the actual flow volume to update
    """WRITE code"""
    # add flowid and flowvolume to the hashtable by calling the function
    add_cms(flowid,flowvolume)
            

#### Checking for anomalies

#### Exercise 20.4b

In [85]:
"""Print the flow IDs that exceeds a threshold"""
threshold = 600
"""WRITE code"""
# iterate through the flowlist and print those flowids having total volume greater than the threshold
for flowid in flowlist:
    volume = query_cms(flowid)
    if(volume>threshold):
        print(flowid, '->', volume)

c0a8a32c0a8a3db2ccc4 -> 4550
c0a8a32ac1001050c9e9 -> 624
c0a8a32ac1001050c9eb -> 624
c0a8a32ac1001050c9ec -> 624


### Some extra info
Checking whether the threshold is exceeded can be done during the update itself and thereby the malicious flow id can be blacklisted in real-time. 

The see how the collisions cause overestimation in CM Sketch, you can rerun the exercise 20.4a by changing the value of m to 16. (You have to make changes in two places - in line 4, and in _hash function.). Then Run 20.4c below to check the difference. (DO NOT run the cell 20.4b again. Keep the result there so that we can compare the results of 20.4a and 20.4b)

#### Exercise 20.4c

In [90]:
"""Print the flow IDs that exceeds a threshold"""
threshold = 600
"""WRITE code"""
# iterate through the flowlist and print those flowids having total volume greater than the threshold
for flowid in flowlist:
    volume = query_cms(flowid)
    if(volume>threshold):
        print(flowid, '->', volume)

c0a8a32c0a8a3db2ccc4 -> 4550
c0a8a32ac1001050c9e9 -> 624
c0a8a32ac1001050c9eb -> 624
c0a8a32ac1001050c9ec -> 624
c0a8a32ac1001050c17f -> 676


We can see that one more flowid is marked as malicious because of overestimation, even though it is not. 

<hr/>
<center>
Continue with the <a href="01_readingframes.ipynb">next notebook</a> in a new browser tab.<br/><br/>
<img src="images/footer.png"/>
</center>